In [ ]:
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import AerSimulator
from qiskit.tools.monitor import job_monitor
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session
from math import pow

In [ ]:
run_simulation = False

In [ ]:
'''
This is a quantum circuit with N qubits. A qubit can be in superposition of 2 states, 
not more.
'''
num_qbits = 7
qc = QuantumCircuit(num_qbits)

In [ ]:
'''
This applies a hadamard gate to actually put the qubits into superposition 
of two states. 
'''
for qubit in range(num_qbits):
    qc.h(qubit)

In [ ]:
'''
measuring collapses the wave function and forces the qubits to choose a value. 
Qubits can only have a superposition of 2 states, so the probabilities collapse
in this case to either 1 or 0. 
'''
qc.measure_all()

In [ ]:
# Visualize the circuit
qc.draw('mpl')

In [ ]:
def simulate(qcircuit): 
    simulator = AerSimulator()
    transpiled_circuit = transpile(qcircuit, simulator)
    result = simulator.run(transpiled_circuit, shots=1).result()
    return result

def run_on_ibm(qcircuit): 
    service = QiskitRuntimeService(channel = "ibm_quantum", token="14c6a65c608138b67d53d32f0959cd73cb06b062f94084c6d7df7b395298cadf3b99f178b17967b3dc14cf6d697fd2fa8642028043cc9a99acd4bed0c4b94158")
    backend = service.get_backend("ibm_nairobi")
    
    transpiled_circuit = transpile(qcircuit, backend)
    
    with Session(backend=backend):
        sampler = Sampler()
        job = sampler.run(transpiled_circuit, shots=1)  
        #job_monitor(job)  # This will display the job's status in real-time
        result = job.result()
        return result
    
def bin_to_dec(bin) -> int: 
    output = 0
    for i in range(len(bin)): 
        if (bin[len(bin)-i-1] == '1'): 
            output += pow(2, i)
    return output

In [ ]:
result = None
if (run_simulation): 
    print('simulating quantum calculation...')
    result = simulate(qc)
else: 
    print('starting actual quantum calculation...')
    run_on_ibm(qc)

In [ ]:
# Get and display the measurement results
counts = result.get_counts()
r = (list(counts.keys())[0]) 
print(r)

In [ ]:
print(bin_to_dec(r)) 